# 🧱 Keras & Data Pipelines (TensorFlow 2)
**Source: TensorFlow in Action – Chapter 3: Keras and data retrieval in TensorFlow 2**

Chapter 3 membahas cara membangun model dengan Keras (Sequential, Functional, Sub-classing API) dan cara men-*stream* data ke model menggunakan `tf.data`, Keras `ImageDataGenerator`, serta paket `tensorflow-datasets`.


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models


## 🧩 Keras Model-Building APIs

**Theory**: Keras yang terintegrasi di TensorFlow menyediakan tiga cara utama membangun model:

- **Sequential API**: untuk arsitektur linear (satu input → lapisan berurutan → satu output).
- **Functional API**: untuk arsitektur kompleks (multi-input, cabang paralel, multi-output).
- **Sub-classing API**: untuk layer/model kustom dengan logika forward pass sendiri.

Setiap API punya trade-off antara kemudahan penulisan dan fleksibilitas desain arsitektur.


In [2]:
import requests

# Download Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
r = requests.get(url)
with open("iris.data", "wb") as f:
    f.write(r.content)

# Load ke pandas
iris_df = pd.read_csv("iris.data", header=None)
iris_df.columns = ["sepallength", "sepalwidth", "petalwidth", "petallength", "label"]

# Map label string → int
iris_df["label"] = iris_df["label"].map({
    "Iris-setosa": 0,
    "Iris-versicolor": 1,
    "Iris-virginica": 2
})

# Shuffle dan centering fitur
iris_df = iris_df.sample(frac=1.0, random_state=4321)
X = iris_df[["sepallength", "sepalwidth", "petalwidth", "petallength"]]
X = X - X.mean(axis=0)
y = tf.one_hot(iris_df["label"], depth=3)

X.head(), y.shape


(     sepallength  sepalwidth  petalwidth  petallength
 31     -0.443333       0.346   -2.258667    -0.798667
 23     -0.743333       0.246   -2.058667    -0.698667
 70      0.056667       0.146    1.041333     0.601333
 100     0.456667       0.246    2.241333     1.301333
 44     -0.743333       0.746   -1.858667    -0.798667,
 TensorShape([150, 3]))

## 🔗 Model A – Sequential API

**Theory**: Sequential API cocok untuk model yang benar-benar berurutan: satu input, beberapa hidden layer `Dense`, dan satu output.

Setiap layer `Dense` menghitung
$
h = \phi(XW + b)
$
di mana \(X\) adalah input, \(W\) bobot, \(b\) bias, dan \(\phi\) fungsi aktivasi (misalnya ReLU atau softmax untuk output klasifikasi).


In [3]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K

K.clear_session()

model_a = Sequential([
    Dense(32, activation="relu", input_shape=(4,)),
    Dense(16, activation="relu"),
    Dense(3, activation="softmax")
])

model_a.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_a.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 3)                 51        
                                                                 
Total params: 739
Trainable params: 739
Non-trainable params: 0
_________________________________________________________________


In [4]:
history_a = model_a.fit(
    X.values,
    y,
    batch_size=64,
    epochs=25,
    verbose=0
)

history_a.history["accuracy"][-1], history_a.history["loss"][-1]


(0.8199999928474426, 0.5480183959007263)

## 🌿 Model B – Functional API + PCA

**Theory**: Functional API memudahkan pembuatan arsitektur model dengan beberapa input,
misalnya input berupa fitur mentah dan input berupa fitur hasil PCA yang kemudian
digabungkan (*concatenate*) sebelum diproses oleh layer lanjutan.

Misalkan terdapat dua representasi tersembunyi dari dua input berbeda, yaitu:

$$
\mathbf{h}_1 \quad \text{dan} \quad \mathbf{h}_2
$$

Kedua representasi tersebut dapat digabungkan menggunakan operasi konkatenasi
sehingga diperoleh representasi baru:

$$
\mathbf{h} = \text{concat}(\mathbf{h}_1, \mathbf{h}_2)
$$

Hasil penggabungan ini selanjutnya diteruskan ke layer Dense berikutnya
untuk proses pembelajaran lebih lanjut.


In [5]:
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Input, Concatenate

# PCA 2 komponen pertama
pca_model = PCA(n_components=2, random_state=4321)
X_pca = pca_model.fit_transform(X.values)

# Functional API
K.clear_session()

inp_raw = Input(shape=(4,), name="raw_features")
inp_pca = Input(shape=(2,), name="pca_features")

h1 = Dense(16, activation="relu")(inp_raw)
h2 = Dense(16, activation="relu")(inp_pca)

h_concat = Concatenate(axis=1)([h1, h2])
h = Dense(16, activation="relu")(h_concat)
out = Dense(3, activation="softmax")(h)

model_b = models.Model(inputs=[inp_raw, inp_pca], outputs=out)
model_b.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_b.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 raw_features (InputLayer)      [(None, 4)]          0           []                               
                                                                                                  
 pca_features (InputLayer)      [(None, 2)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 16)           80          ['raw_features[0][0]']           
                                                                                                  
 dense_1 (Dense)                (None, 16)           48          ['pca_features[0][0]']           
                                                                                              

In [6]:
history_b = model_b.fit(
    [X.values, X_pca],
    y,
    batch_size=64,
    epochs=10,
    verbose=0
)

history_b.history["accuracy"][-1], history_b.history["loss"][-1]


(0.6866666674613953, 0.8173895478248596)

## 🛠️ Model C – Custom Layer (Sub-classing API)

**Theory**: Sub-classing API digunakan ketika diperlukan pembuatan layer kustom
dengan formulasi yang berbeda dari layer `Dense` standar. Pendekatan ini
memberikan fleksibilitas penuh dalam mendefinisikan perilaku layer, baik dari
sisi parameter maupun proses komputasinya.

Pada contoh yang dibahas, fungsi hidden layer dimodifikasi menjadi:

$$
h = \phi(XW + b + b_{\text{mul}})
$$

di mana \(W\) merupakan bobot, \(b\) adalah bias utama, dan \(b_{\text{mul}}\)
merupakan *multiplicative bias* tambahan yang ikut dipelajari selama proses
training bersama parameter lainnya. Fungsi aktivasi \(\phi(\cdot)\) digunakan
untuk menghasilkan representasi non-linear pada output layer.

Layer kustom seperti ini diimplementasikan dengan melakukan sub-class terhadap
`tf.keras.layers.Layer`, kemudian mendefinisikan metode `build` untuk
menginisialisasi parameter dan metode `call` untuk mendeskripsikan proses
*forward pass* saat data dilewatkan melalui layer tersebut.


In [7]:
from tensorflow.keras import layers

class MulBiasDense(layers.Layer):
    def __init__(self, units=32, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = activation
    
    def build(self, input_shape):
        # create weights
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="w"
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="glorot_uniform",
            trainable=True,
            name="b"
        )
        self.b_mul = self.add_weight(
            shape=(self.units,),
            initializer="glorot_uniform",
            trainable=True,
            name="b_mul"
        )
    
    def call(self, inputs):
        out = tf.matmul(inputs, self.w) + self.b + self.b_mul
        if self.activation is not None:
            return tf.keras.activations.get(self.activation)(out)
        return out

K.clear_session()

inp_c = Input(shape=(4,))
h_c = MulBiasDense(units=32, activation="relu")(inp_c)
h_c = MulBiasDense(units=16, activation="relu")(h_c)
out_c = Dense(3, activation="softmax")(h_c)

model_c = models.Model(inputs=inp_c, outputs=out_c)
model_c.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_c.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 mul_bias_dense (MulBiasDens  (None, 32)               192       
 e)                                                              
                                                                 
 mul_bias_dense_1 (MulBiasDe  (None, 16)               544       
 nse)                                                            
                                                                 
 dense (Dense)               (None, 3)                 51        
                                                                 
Total params: 787
Trainable params: 787
Non-trainable params: 0
_________________________________________________________________


In [8]:
history_c = model_c.fit(
    X.values,
    y,
    batch_size=64,
    epochs=25,
    verbose=0
)

history_c.history["accuracy"][-1], history_c.history["loss"][-1]


(0.8533333539962769, 0.4686679542064667)

## 🚚 tf.data API untuk Input Pipeline

**Theory**: `tf.data` menyediakan mekanisme yang fleksibel dan efisien untuk
membangun *input pipeline*, mulai dari membaca data dari file atau CSV,
menerapkan transformasi seperti decoding, resizing, dan normalisasi,
hingga melakukan pengacakan (*shuffle*) dan pembagian data ke dalam batch
sebelum diproses oleh model.

Pipeline input yang umum digunakan diawali dengan pembuatan objek
`tf.data.Dataset` dari sumber data, seperti file atau CSV. Selanjutnya,
fungsi `map` digunakan untuk membaca dan melakukan *preprocessing* data,
misalnya dengan `tf.io.read_file`, `tf.image.decode_png`, dan
`tf.image.resize`. Setelah itu, data diacak menggunakan `shuffle`,
dibagi ke dalam batch dengan `batch`, dan dioptimalkan menggunakan
`prefetch` agar proses input data dan komputasi model dapat berjalan
secara paralel dan lebih efisien.


In [9]:
# dataset sederhana dari X, y (Iris)
ds = tf.data.Dataset.from_tensor_slices((X.values.astype("float32"), y))

ds = ds.shuffle(buffer_size=150, reshuffle_each_iteration=True)
ds = ds.batch(32).prefetch(tf.data.AUTOTUNE)

for batch_x, batch_y in ds.take(1):
    print(batch_x.shape, batch_y.shape)


(32, 4) (32, 3)


## 🌉 Keras ImageDataGenerator & `tensorflow-datasets`

**Theory**: Keras menyediakan beberapa utilitas untuk mempermudah pengelolaan
data pada proses pelatihan model, khususnya untuk data citra dan dataset
standar yang telah tersedia.

`ImageDataGenerator` berfungsi sebagai generator data citra yang mendukung
berbagai teknik augmentasi, seperti rotasi, *flip*, dan *rescaling*, serta
menyediakan mekanisme pemuatan data langsung dari struktur folder atau
`DataFrame`. Pendekatan ini sangat cocok digunakan untuk tahap prototyping
cepat pada permasalahan pengolahan citra.

Selain itu, paket `tensorflow-datasets (tfds)` menyediakan akses mudah ke
berbagai dataset standar, seperti CIFAR-10, Caltech101, dan IMDb, yang dapat
dimuat hanya dengan satu perintah `tfds.load`. Dataset yang dihasilkan sudah
berbentuk `tf.data.Dataset` sehingga dapat langsung di-*batch* dan digunakan
dalam pelatihan model Keras.

Kedua pendekatan tersebut dapat diintegrasikan langsung dengan model Keras
melalui fungsi `model.fit(generator_or_dataset, ...)`, tanpa memerlukan
penulisan loop pelatihan secara manual.
